# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one (or multiple)
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using Unitful
using UnitfulAtomic
using PseudoPotentialData

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    # Use AtomsBuilder to setup aluminium cubic unit cell (4 Al atoms)
    # with provided lattice constant, see AtomsBase integration for details.
    unit_cell = bulk(:Al; a=7.65339u"bohr", cubic=true)
    supercell = unit_cell * (repeat, 1, 1)  # Make a supercell

    # Select standard pseudodojo pseudopotentials, construct an LDA model, discretize
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    pseudopotentials = PseudoFamily("dojo.nc.sr.lda.v0_4_1.oncvpsp3.standard.upf")
    model = model_DFT(supercell; pseudopotentials, functionals=LDA(),
                      temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As expected we obtain the unit cell for `repeat=1`:

In [2]:
aluminium_setup(1)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (24, 24, 24), 13824 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [7.65339   , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 448.29 Bohr³
    
        atoms                : Al₄
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.u

and 5-fold as large supercell with `repeat=5`:

In [3]:
aluminium_setup(5)

PlaneWaveBasis discretization:
    architecture         : DFTK.CPU()
    num. mpi processes   : 1
    num. julia threads   : 1
    num. DFTK  threads   : 1
    num. blas  threads   : 2
    num. fft   threads   : 1

    Ecut                 : 7.0 Ha
    fft_size             : (96, 24, 24), 55296 total points
    kgrid                : MonkhorstPack([2, 2, 2])
    num.   red. kpoints  : 8
    num. irred. kpoints  : 8

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [38.267    , 0         , 0         ]
                               [0         , 7.65339   , 0         ]
                               [0         , 0         , 7.65339   ]
        unit cell volume     : 2241.5 Bohr³
    
        atoms                : Al₂₀
        atom potentials      : ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.upf")
                               ElementPsp(Al, "/home/runner/.julia/artifacts/1ea71a84cf375286564538a9cab789991f4bf1f4/Al.

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [4]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355377439559                   -1.10    5.9    187ms
  2   -9.356741224175       -2.87       -1.42    1.0    118ms
  3   -9.356974447046       -3.63       -2.74    3.4    122ms
  4   -9.357012035184       -4.42       -3.07    5.6    476ms
  5   -9.357012275855       -6.62       -3.20    2.6    107ms
  6   -9.357012403363       -6.89       -3.36    1.0   81.3ms
  7   -9.357012476053       -7.14       -3.50    1.1   83.5ms
  8   -9.357012514849       -7.41       -3.64    1.1   89.2ms
  9   -9.357012540063       -7.60       -3.91    1.0   81.9ms
 10   -9.357012547415       -8.13       -4.30    2.8    112ms


In [5]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Eigensolver not converged
│   n_iter =
│    8-element Vector{Int64}:
│      9
│      9
│      8
│     11
│     13
│      8
│      5
│      8
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:76
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -18.78029668043                   -0.97    8.9    669ms
  2   -18.79183594946       -1.94       -1.30    1.4    273ms
  3   -18.79228842607       -3.34       -2.81    2.5    289ms
  4   -18.79240716122       -3.93       -2.95    8.8    677ms
  5   -18.79240840026       -5.91       -3.21    1.1    228ms
  6   -18.79240856006       -6.80       -3.54    1.5    245ms
  7   -18.79240859299       -7.48       -4.22    2.5    279ms


In [6]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55468633604                   -0.84    9.4    2.20s
  2   -37.56020560484       -2.26       -1.19    1.6    769ms
  3   -37.56178755845       -2.80       -2.55    6.2    1.19s
  4   -37.56452945776       -2.56       -2.43    5.6    1.68s
  5   -37.56452349148   +   -5.22       -2.38   11.5    1.27s
  6   -37.56455342808       -4.52       -3.39    1.0    751ms
  7   -37.56455380783       -6.42       -3.64    9.2    1.35s
  8   -37.56455405621       -6.60       -4.10    2.0    1.12s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [7]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -9.355329401760                   -1.09    6.8    237ms
  2   -9.356781921168       -2.84       -1.91    1.0   76.9ms
  3   -9.356997254938       -3.67       -2.82    4.4    144ms
  4   -9.356995590749   +   -5.78       -2.64    3.4    139ms
  5   -9.357012066343       -4.78       -3.44    1.1   75.4ms
  6   -9.357012481340       -6.38       -3.90    3.0    122ms
  7   -9.357012546458       -7.19       -4.61    2.6    112ms


In [8]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -37.55505765972                   -0.84    9.9    2.22s
  2   -37.54562063089   +   -2.03       -1.59    1.1    649ms
  3   -22.06871342448   +    1.19       -0.47    8.0    1.61s
  4   -37.56208506341        1.19       -1.99    8.9    2.05s
  5   -37.55339425325   +   -2.06       -1.89    3.8    1.08s
  6   -37.54448629854   +   -2.05       -1.90    3.4    983ms
  7   -37.48493791865   +   -1.23       -1.61    5.2    1.15s
  8   -37.56393837184       -1.10       -2.38    4.9    1.40s
  9   -37.56447699093       -3.27       -2.89    2.8    957ms
 10   -37.56453824230       -4.21       -3.14    4.4    1.27s
 11   -37.56454137087       -5.50       -3.18    2.8    1.08s
 12   -37.56455242779       -4.96       -3.64    1.6    1.18s
 13   -37.56455368207       -5.90       -3.92    3.2    1.05s
 14   -37.56455401848       -6.47       -4.37    2.2    907ms


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.